# Delivering Slides with Agents (GPT-4) y DALL-E-3.

This notebook is an updated version of the openai agents called assistants [Assistants API](https://platform.openai.com/docs/assistants/overview) with  GPT-4, and DALL·E-3 on delivering slides. 

Making slides is hardworking and could be tedious and time demanding.  Additionally extract relevan information and articulate effectively words on slides could be challenging and demanding.

This notebook demonstrates how to enable the assistants to work for you doing a powerpoint presentation or google sheets saving valuable time.

## 0. Settings

In [1]:
from IPython.display import display, Image
from openai import OpenAI
import os
import pandas as pd
import json
import io
from PIL import Image
import requests

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", ""))

#Lets import some helper functions for assistants from https://cookbook.openai.com/examples/assistants_api_overview_python

# Function to display a JSON File
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# you can change the model if you are on text only
def submit_message(assistant_id, thread, user_message, model=None, file_ids=None):
    if model: 
        client.beta.assistants.update(assistant_id=assistant_id, model=model)

    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
        }
    
    if file_ids:
        #client.beta.assistants.update(assistant_id=assistant_id, tool_resources={"code_interpreter":{"file_ids":file_ids}})        
        params['attachments'] = [{ "file_id": fid, "tools": [{"type": "code_interpreter"}]} for fid in file_ids]
        
    client.beta.threads.messages.create(**params) # create a thread
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

## 1. Creating the content

Let's get the financial data, the data is in json format and represents a fake company quarter data. Let's take a look before insights

In [2]:
financial_data_path = 'data/data_financiera.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)

,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


The dataset has quarterly revenue across the years, quarters and distribution channels. We will create an assistant that can personally create a good powerpoint presentation.

#### 1.1 Example Assistants use

##### 1.1.1 First, we need to upload our file so our Assistant can access it.

In [3]:
file = client.files.create(
  file=open('data/data_financiera.json',"rb"),
  purpose='assistants',
)

#####  1.1.2 Next we need to create the assistant

Now, we're ready to create our Assistant. Let's instruct our assistant to act as a data scientist, take some queries to run the necessary code to output the visualization. The instruction is similar to the chatcompletion endpoint, and this will serve as help to the assistant. We also will turn on the tool of Code Interpreter enable our assistant to code.  Finally we can specify to the assistant any files we want to use but i will prefer to do that when the task is created; let's create the assistant.

In [4]:
assistant = client.beta.assistants.create(
  instructions="Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.",
  name = "data_science_ppt_creator",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}], 
  # tool_resources={
  #     "code_interpreter": {
  #         "file_ids":[file.id]
  #     }
  # }
)

#####  1.1.3 Next we need to create the thread

Now, let's create the thread as our request of the assistant to calculate quarter profits and plot the distribution channel over time.  

We will calculate the profits and later will tell the LLM to plot the profits by distribution channel over time. The assistant will automatically calculate the profit each quarter and also will combining quarter and year in a new column without telling anything directly, we could also tell the color of lines.

In [5]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Del archivo .json cargado: Calcular la utilidad (ingreso - gasto)  por cuarto y por año, visualizalo en un grafico de lineas a traves de la distribución de canales, donde los colores de las lineas son rojo, verde, and azul claro",
      "attachments": [{"file_id": file.id, "tools": [{"type": "code_interpreter"}]}]
    }
  ]
)

##### 1.1.4 Run the thread

With everything ready, let's run the thread

In [6]:
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [7]:
#client.beta.threads.delete(run.thread_id)

##### 1.1.5 Loop over the message list

We can start to look over the list of created messages.

NOTE:  For some actions this could take some minutes.

In [8]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [9]:
messages

SyncCursorPage[Message](data=[Message(id='msg_Cy2OuCUv7yni0ex9Z52127Ze', assistant_id='asst_zrdWolhagCAcb8uyrRjXu2TR', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Antes de poder calcular la utilidad y visualizarla, tendré que cargar y examinar el contenido del archivo JSON para entender su estructura. Comencemos leyendo el archivo para ver qué tipo de datos contiene.'), type='text')], created_at=1714117747, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_I9slGFKNq4dFRaPDEgNKzEgy', status=None, thread_id='thread_KbYDPm3UWvRuUizB8wc9fbja'), Message(id='msg_jLsl2SMTXumCW78q4t1ySNgB', assistant_id=None, attachments=[Attachment(file_id='file-lN3ysRA1DthMvoWNgJxXVViU', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Del archivo .json cargado: Calcular la utilidad (ingreso - gasto)  por cuarto y

In [10]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #See if image has been created
        messages.data[0].content[0].image_file
        #Sleep to make sure run has completed
        time.sleep(5)
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')

Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


In [12]:
# to stop the runs
#client.beta.threads.runs.cancel(run_id=run.id, thread_id=thread.id)

##### 1.1.6 Let's loop over the mesages

Let's see the messages the Assistant added.  As you see everything is intended to be in blocks, for example if we want to access the imagefile it will be message.content[0].image_file.file_id.

In [11]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]

[ImageFileContentBlock(image_file=ImageFile(file_id='file-Tx9ZdN7wOYySwk415uoksJ3o'), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value="Ahora que tengo los datos en un DataFrame de pandas, calcularé la utilidad (ingreso - gasto) por trimestre y año, agrupándolos por 'Distribution channel'. A continuación, voy a generar una visualización en un gráfico de líneas para mostrar la distribución de utilidades a través de los diferentes canales con colores específicos."), type='text'),
 TextContentBlock(text=Text(annotations=[], value='Parece que los datos JSON tienen una estructura tabular donde `Year`, `Quarter`, `Channel`, `Revenue`, y `Expenses` podrían ser columnas clave para calcular las utilidades. Sin embargo, dado que solo mostré los primeros 1000 caracteres, no tengo toda la información para confirmar esto. \n\nVoy a continuar explorando la estructura completa del archivo y luego cargaré los datos en un DataFrame de pandas para proceder con los cálculos y la visu

We can see that the last message (latest message is shown first) from the assistant contains the image file we are looking for. An interesting note here is that the Assistant was able to attempt several times to parse the JSON data, as the first parsing was unsuccessful, demonstrating the assistant's adaptability.

In [12]:
# Quick helper function to convert our output file to a png
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)

In [13]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "imgs/grafica_financiera.png"
convert_file_to_png(plot_file_id,image_path)

#Upload
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)


Let's load in the plot!

![The Image](imgs/grafica_financiera.png)

Nice! So, with just one sentence, we were able to have our assistant use code interpreter to
calculate the profitability, and graph the three lineplots of the various distribution channels.<br><br>
Now we have a nice visual for our slide, but we want some insights to go along with it.

## 2. Generating insights

In the same thread, let's use the image and let the assistant to generate insights.  Our assistant will retreive the message history to give us the pros and cons of concise information of the takeaway. 

Let's use our helper function in line 1 that will keep everything in order using the same thread (or other, in this case the same).

In [14]:
submit_message(assistant_id=assistant.id, 
               thread=thread, 
               model="gpt-4-turbo", 
               file_ids=None,
               user_message="Escribe dos oraciones de largo aproximadamente medio (~20-30 palabras por oracion) de \
lo mas importante captado de la gráfica que acabas de crear. Esta grafica será añadida para un  conjunto de diapositivas, \
y ellas deberian representar acerca del '¿Que hay detras de los datos?'.")

Run(id='run_ayPA0C6TMyvfZZTm9BhWQKmK', assistant_id='asst_zrdWolhagCAcb8uyrRjXu2TR', cancelled_at=None, completed_at=None, created_at=1714117827, expires_at=1714118427, failed_at=None, incomplete_details=None, instructions='Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_KbYDPm3UWvRuUizB8wc9fbja', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

Now, once the run has completed, we can see the latest message

In [16]:
# Hard coded wait for a response, as the assistant may iterate on the bullets.
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)

El gráfico demuestra que la utilidad generada por las ventas en línea ha mostrado una tendencia generalmente positiva y consistente a lo largo del tiempo, indicando un crecimiento estable y fiable en este canal. Por otro lado, las fluctuaciones en los canales de ventas directas y socios minoristas sugieren una variabilidad en la efectividad de estas estrategias a lo largo de los diferentes trimestres y años, lo que podría señalar áreas de oportunidad para optimización y mejora.


Cool! So our assistant was able to identify the noteworthy growth in Online Sales profit, and infer that this shows the importance of a large digital presence. Now let's get a compelling title for the slide.

In [17]:
submit_message(assistant_id=assistant.id, thread=thread, model="gpt-4-turbo", user_message="Dada la gráfica y las viñetas que has creado,\
 piense en un titulo muy breve para una diapositiva. Debe reflejar solo las ideas principales que se te ocurrieron."
)

Run(id='run_vXiZnwDLaG52QMYfRGrTtIlh', assistant_id='asst_zrdWolhagCAcb8uyrRjXu2TR', cancelled_at=None, completed_at=None, created_at=1714117867, expires_at=1714118467, failed_at=None, incomplete_details=None, instructions='Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_KbYDPm3UWvRuUizB8wc9fbja', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

And the title is.....-:

In [18]:
#Wait as assistant may take a few steps
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)

"Evolución y Oportunidades en la Utilidad de Diferentes Canales de Distribución"


## 3. DALL·E-3 title image

Now let's use Dall-E-3 to generate the slide first page image of the presentation. <br><br>

In [19]:
company_summary = "MiEmpresaFinanciera es una empresa promiennte de hardware que manufactura y vende procesadores, tarjetas gráficas\
 y otras piezas esenciales de hardware de computadoras."

In [20]:
response = client.images.generate(
  model='dall-e-3',
  prompt=f"Dado este resumen de la empresa {company_summary}, crea una foto inspiradora que \
  muestre el crecimiento y el camino a seguir. Esto se utilizará en una reunión trimestral \
  de planificación financiera",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url

For adding the image to the thread, first, we can save the image locally, then upload it to the assistants API using the `File` upload endpoint. Let's also take a look at our image

In [21]:
dalle_img_path = 'imgs/imagen_dalle3.png'
img = requests.get(image_url)

#Save locally
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#Upload
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)

![dalle3_img](imgs/imagen_dalle3.png "Titulo PPT")

## 4. Creating the slides

We now have all the content we need to create the slides. While we could simply add a message asking for slides, but let's instead give the assistant a slide template, using the `python-pptx` library, to use. This will ensure we get a deck in the style we want. See the `Extensions` section at the end of the notebook for notes on creating the template.

Below, title_template is a string that contains the code that will be supplied to the assistant code interpreter and later we will use in the assistant message to generate powerpoint slides

In [22]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left side of the slide with a margin at the top and bottom
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text box positioned higher
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle text box
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Define placeholders
image_path = data_vis_img
title_text = "Maximizando Beneficios: El Dominio de las Ventas Activas & Optimización de Venta Directa"
bullet_points = "• Las ventas online consistentemente llevan a ganancias en los cuartos, indicando una fuerte presencia de marketing digital.\n• Las ventas directas muestran fluctuaciones, sugiriendo desempeño variable y la necesidas de apuntar mejoras en el canal."

# Add image placeholder on the left side of the slide
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text spanning the whole width
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add hardcoded "Key Insights" text and bullet points
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""



Let's set a few quick variables for our slides. We want the company name, MiEmpresaFinanciera, to be on the title slide, and the title of the presentation should 'Quartlerly financial planning metting, Q3, 2023'.

In [23]:
title_text = "MiEmpresaFinanciera"
subtitle_text = "Reunion de Planificacion del Q3, 2023"


And for the data slide, we have:

Here we have a template to create a Title Slide. The template below was created by uploading the image of a desirable title slide to GPT-V, and asking for the `python-pptx` code to create that template. The inputs to the template are the image_path, title_text, and subtitle_text.

In [24]:
run = submit_message(assistant_id=assistant.id, thread=thread, model="gpt-4-turbo", user_message=f"IMPORTANTE: Utilizar {dalle_file.filename} \
con key_id {dalle_file.id}; utilizar {plot_file.filename} con key_id {plot_file.id} en filmina 2. Utilice la plantilla de código incluida \
para crear una diapositiva PPTX que siga el formato de la plantilla, pero utilice la imagen, el nombre/título de la empresa y el nombre/subtítulo \
del documento incluidos:{title_template}. IMPORTANTE: utilice el archivo de imagen incluido en este mensaje como imagen image_path en esta primera \
diapositiva y utilice el nombre de la empresa {title_text} como variable title_text y  utilice subtitle_text {subtitle_text} como \
variable subtitle_text. A CONTINUACIÓN, cree una SEGUNDA diapositiva usando la siguiente plantilla de código: {data_vis_template} para crear \
una diapositiva PPTX que siga el formato de la plantilla, pero use el nombre/título de la empresa y el nombre/subtítulo del \
documento incluidos: {data_vis_template}. IMPORTANTE: use la imagen del diagrama de líneas, que es la segunda imagen adjunta en \
este mensaje, que creó anteriormente en thread como imagen data_vis_img, y use el título de visualización de datos que creó anteriormente \
para la variable title_text, y las viñetas de conocimientos que creó anteriormente para la variable bullet_points. \
Imprima estas DOS DIAPOSITIVAS como un archivo .pptx. Asegúrese de que el resultado sean dos diapositivas, y que cada diapositiva coincida \
con la plantilla respectiva proporcionada en este mensaje.",
file_ids=[plot_file.id, dalle_file.id]
)
run

Run(id='run_AtajmVIBa1RjYMG1kvIdRVJR', assistant_id='asst_zrdWolhagCAcb8uyrRjXu2TR', cancelled_at=None, completed_at=None, created_at=1714117945, expires_at=1714118545, failed_at=None, incomplete_details=None, instructions='Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_KbYDPm3UWvRuUizB8wc9fbja', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [25]:
# cancelar la corrida
#client.beta.threads.runs.cancel(run_id=run.id, thread_id=thread.id)

In [26]:
#May take 1-3 mins
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)
        break
    except Exception as e:
        print("Assistant still working on PPTX...")
        time.sleep(10)

Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Successfully retrieved pptx_id: file-PrOc77t0D8k5Qc9Fmk17SK83


In [27]:
# para detener la tarea
#client.beta.threads.delete(thread_id=thread.id)

In [30]:
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("data/filminas_creadas.pptx", "wb") as f:
    f.write(file_obj.getbuffer())

Now, we have a PPTX file saved with all of our created content!. <br>

Let's look at the screenshots of the .pptx we just created using JUST the assistants API and DALL·E-3. DALL-E-3 has not yet in the Assistants API a seed parameter, so the DALL·E-3 image and wordings will be slightly different from what you see when you run this notebook, due to the non-determinism of LLMs, but the outputs should be directionally the same.

The title slide:

![The Image](imgs/imagen_dalle3.png)

And the data slide:

![The Image](imgs/grafica_financiera.png)

## 5. Conclusion

Woo! While these slides could use some formatting tweaks, we have made some great content using the Assistants API, GPT-4 and DALL·E-3. We were able to take a `.csv` file with financial data, and use our assisant to calculate profit by quarter across distribution channels, plot the results, identify insights and key takeaways from the visualization, and create a summarative title. And, given just a description of our company, MiEmpresaFinanciera, we used DALL·E-3 to make an awesome title image. <br><br>
While we are still a ways away from entirely automating this process without a human in the loop, hopefully this notebook can make the slide creation process a bit easier for you. More importantly, this notebook can ideally give you a glimpse into the potential of the assistants API! We're excited to see what you build.

## 6. Extensions

- When  DALL·E-3 is incorporated in the Assistants API, we will have the ability to request the generated title image within the thread. 
- GPT-4-Vision is not yet supported in the Assistants API, but could have been used to gather insights from the line plot image.
- GPT-4-Vision was used to generate the `python-pptx` template included in this recipe, so a potential extension project could be demonstrating best practices around converting images to slide templates.

In [58]:
#client.beta.threads.delete(thread_id=)
#client.beta.threads.runs.cancel(run_id=)
#client.beta.assistants.delete(assistant_id=asst_VHenAVYAorsqJ9bgtXDh1603)

AssistantDeleted(id='asst_VHenAVYAorsqJ9bgtXDh1603', deleted=True, object='assistant.deleted')